In [1]:
import argparse
import logging
import socket
import sys
import time
import random
import os
import pickle
import torch
import numpy as np
from sys import exit
from collections import defaultdict, namedtuple
import itertools
import json
import math
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import matplotlib.pyplot as plt

In [2]:
def prepare_data(path, subset='/train/', sample=1.0, goals=True):
    """ Prepares the train/val scenes and corresponding goals 
    
    Parameters
    ----------
    subset: String ['/train/', '/val/']
        Determines the subset of data to be processed
    sample: Float (0.0, 1.0]
        Determines the ratio of data to be sampled
    goals: Bool
        If true, the goals of each track are extracted
        The corresponding goal file must be present in the 'goal_files' folder
        The name of the goal file must be the same as the name of the training file

    Returns
    -------
    all_scenes: List
        List of all processed scenes
    all_goals: Dictionary
        Dictionary of goals corresponding to each dataset file.
        None if 'goals' argument is False.
    Flag: Bool
        True if the corresponding folder exists else False.
    """

    ## Check if folder exists
    if not os.path.isdir(path + subset):
        if 'train' in subset:
            print("Train folder does NOT exist")
            exit()
        if 'val' in subset:
            print("Validation folder does NOT exist")
            return None, None, False

    ## read goal files
    all_goals = {}
    all_scenes = []

    ## List file names
    # files = [f.split('.')[-2] for f in os.listdir(path + subset) if f.endswith('.ndjson')]
    files = [os.path.splitext(os.path.basename(f))[0] for f in os.listdir(path + subset) if f.endswith('.ndjson')]
    #此处应该考虑每个文件的scaler不同。需要把scaler一起读出来。
    ## Iterate over file names
    for file in files:
        reader = Reader(path + subset + file + '.ndjson', scene_type='paths')
        ## Necessary modification of train scene to add filename
        scene = [(file, s_id, s) for s_id, s in reader.scenes(sample=sample)]
        if goals:
            goal_dict = pickle.load(open('goal_files/' + subset + file +'.pkl', "rb"))
            ## Get goals corresponding to train scene
            all_goals[file] = {s_id: [goal_dict[path[0].pedestrian] for path in s] for _, s_id, s in scene}
        all_scenes += scene

    if goals:
        return all_scenes, all_goals, True
    return all_scenes, None, True


In [3]:
def main(epochs=100):
    parser = argparse.ArgumentParser()
    parser.add_argument('--epochs', default=epochs, type=int,
                        help='number of epochs')
    parser.add_argument('--save_every', default=5, type=int,
                        help='frequency of saving model (in terms of epochs)')
    parser.add_argument('--obs_length', default=25, type=int,
                        help='observation length')
    parser.add_argument('--pred_length', default=25, type=int,
                        help='prediction length')
    parser.add_argument('--start_length', default=0, type=int,
                        help='starting time step of encoding observation')
    parser.add_argument('--batch_size', default=8, type=int)
    parser.add_argument('--lr', default=1e-3, type=float,
                        help='initial learning rate')
    parser.add_argument('--step_size', default=10, type=int,
                        help='step_size of lr scheduler')
    parser.add_argument('-o', '--output', default=None,
                        help='output file')
    parser.add_argument('--disable-cuda', action='store_true',
                        help='disable CUDA')
    parser.add_argument('--path', default='trajdata',
                        help='glob expression for data files')
    parser.add_argument('--goals', action='store_true',
                        help='flag to consider goals of pedestrians')
    parser.add_argument('--loss', default='pred', choices=('L2', 'pred'),
                        help='loss objective, L2 loss (L2) and Gaussian loss (pred)')
    parser.add_argument('--type', default='vanilla',
                        choices=('vanilla', 'occupancy', 'directional', 'social', 'hiddenstatemlp',
                                 'nn', 'attentionmlp', 'nn_lstm', 'traj_pool'),
                        help='type of interaction encoder')
    parser.add_argument('--sample', default=1.0, type=float,
                        help='sample ratio when loading train/val scenes')
    parser.add_argument('--seed', type=int, default=42)

    # Augmentations
    parser.add_argument('--augment', action='store_true',
                        help='perform rotation augmentation')
    parser.add_argument('--normalize_scene', action='store_true',
                        help='rotate scene so primary pedestrian moves northwards at end of observation')
    parser.add_argument('--augment_noise', action='store_true',
                        help='flag to add noise to observations for robustness')
    parser.add_argument('--obs_dropout', action='store_true',
                        help='perform observation length dropout')

    # Loading pre-trained models
    pretrain = parser.add_argument_group('pretraining')
    pretrain.add_argument('--load-state', default=None,
                          help='load a pickled model state dictionary before training')
    pretrain.add_argument('--load-full-state', default=None,
                          help='load a pickled full state dictionary before training')
    pretrain.add_argument('--nonstrict-load-state', default=None,
                          help='load a pickled state dictionary before training')

    # Sequence Encoder Hyperparameters
    hyperparameters = parser.add_argument_group('hyperparameters')
    hyperparameters.add_argument('--hidden-dim', type=int, default=128,
                                 help='LSTM hidden dimension')
    hyperparameters.add_argument('--coordinate-embedding-dim', type=int, default=64,
                                 help='coordinate embedding dimension')
    hyperparameters.add_argument('--pool_dim', type=int, default=256,
                                 help='output dimension of interaction vector')
    hyperparameters.add_argument('--goal_dim', type=int, default=64,
                                 help='goal embedding dimension')

    # Grid-based pooling
    hyperparameters.add_argument('--cell_side', type=float, default=0.6,
                                 help='cell size of real world (in m) for grid-based pooling')
    hyperparameters.add_argument('--n', type=int, default=12,
                                 help='number of cells per side for grid-based pooling')
    hyperparameters.add_argument('--layer_dims', type=int, nargs='*', default=[512],
                                 help='interaction module layer dims for gridbased pooling')
    hyperparameters.add_argument('--embedding_arch', default='one_layer',
                                 help='interaction encoding arch for gridbased pooling')
    hyperparameters.add_argument('--pool_constant', default=0, type=int,
                                 help='background value (when cell empty) of gridbased pooling')
    hyperparameters.add_argument('--norm_pool', action='store_true',
                                 help='normalize the scene along direction of movement during grid-based pooling')
    hyperparameters.add_argument('--front', action='store_true',
                                 help='flag to only consider pedestrian in front during grid-based pooling')
    hyperparameters.add_argument('--latent_dim', type=int, default=16,
                                 help='latent dimension of encoding hidden dimension during social pooling')
    hyperparameters.add_argument('--norm', default=0, type=int,
                                 help='normalization scheme for input batch during grid-based pooling')

    # Non-Grid-based pooling
    hyperparameters.add_argument('--no_vel', action='store_true',
                                 help='flag to not consider relative velocity of neighbours')
    hyperparameters.add_argument('--spatial_dim', type=int, default=32,
                                 help='embedding dimension for relative position')
    hyperparameters.add_argument('--vel_dim', type=int, default=32,
                                 help='embedding dimension for relative velocity')
    hyperparameters.add_argument('--neigh', default=4, type=int,
                                 help='number of nearest neighbours to consider')
    hyperparameters.add_argument('--mp_iters', default=5, type=int,
                                 help='message passing iterations in NMMP')

    # Collision Loss
    hyperparameters.add_argument('--col_wt', default=0., type=float,
                                 help='collision loss weight')
    hyperparameters.add_argument('--col_distance', default=0.2, type=float,
                                 help='distance threshold post which collision occurs')
    args = parser.parse_args()

    # Set seed for reproducibility
    torch.manual_seed(args.seed)
    random.seed(args.seed)

    # Define location to save trained model
    if not os.path.exists('OUTPUT_BLOCK/{}'.format(args.path)):
        os.makedirs('OUTPUT_BLOCK/{}'.format(args.path))
    if args.goals:
        args.output = 'OUTPUT_BLOCK/{}/lstm_goals_{}_{}.pkl'.format(args.path, args.type, args.output)
    else:
        args.output = 'OUTPUT_BLOCK/{}/lstm_{}_{}.pkl'.format(args.path, args.type, args.output)

    # configure logging
    if args.load_full_state:
        file_handler = logging.FileHandler(args.output + '.log', mode='a')
    else:
        file_handler = logging.FileHandler(args.output + '.log', mode='w')
    file_handler.setFormatter(jsonlogger.JsonFormatter('%(message)s %(levelname)s %(name)s %(asctime)s'))
    stdout_handler = logging.StreamHandler(sys.stdout)
    logging.basicConfig(level=logging.INFO, handlers=[stdout_handler, file_handler])
    logging.info({
        'type': 'process',
        'argv': sys.argv,
        'args': vars(args),
        'version': version,
        'hostname': socket.gethostname(),
    })

    # refactor args for --load-state
    # loading a previously saved model
    args.load_state_strict = True
    if args.nonstrict_load_state:
        args.load_state = args.nonstrict_load_state
        args.load_state_strict = False
    if args.load_full_state:
        args.load_state = args.load_full_state

    # add args.device
    args.device = torch.device('cpu')
    if not args.disable_cuda and torch.cuda.is_available():
        args.device = torch.device('cuda')

    args.path = 'DATA_BLOCK/' + args.path
    # Prepare data
    train_scenes, train_goals, _ = prepare_data(args.path, subset='/train/', sample=args.sample, goals=args.goals)
    val_scenes, val_goals, val_flag = prepare_data(args.path, subset='/val/', sample=args.sample, goals=args.goals)

    # pretrained pool model (if any)
    pretrained_pool = None

    # # create interaction/pooling modules
    # pool = None
    # if args.type == 'hiddenstatemlp':
    #     pool = HiddenStateMLPPooling(hidden_dim=args.hidden_dim, out_dim=args.pool_dim,
    #                                  mlp_dim_vel=args.vel_dim)
    # elif args.type == 'attentionmlp':
    #     pool = AttentionMLPPooling(hidden_dim=args.hidden_dim, out_dim=args.pool_dim,
    #                                mlp_dim_spatial=args.spatial_dim, mlp_dim_vel=args.vel_dim)
    # elif args.type == 'nn':
    #     pool = NearestNeighborMLP(n=args.neigh, out_dim=args.pool_dim, no_vel=args.no_vel)
    # elif args.type == 'nn_lstm':
    #     pool = NearestNeighborLSTM(n=args.neigh, hidden_dim=args.hidden_dim, out_dim=args.pool_dim)
    # elif args.type == 'traj_pool':
    #     pool = TrajectronPooling(hidden_dim=args.hidden_dim, out_dim=args.pool_dim)
    # elif args.type != 'vanilla':
    #     pool = GridBasedPooling(type_=args.type, hidden_dim=args.hidden_dim,
    #                             cell_side=args.cell_side, n=args.n, front=args.front,
    #                             out_dim=args.pool_dim, embedding_arch=args.embedding_arch,
    #                             constant=args.pool_constant, pretrained_pool_encoder=pretrained_pool,
    #                             norm=args.norm, layer_dims=args.layer_dims, latent_dim=args.latent_dim)
    # hyper parameters
    input_size = 8  # frame, car_id, x, y, xVelocity, yVelocity, dx, dy, dvx, dvy
    hidden_size = 8  # LSTM hidden size
    num_layers = 1  # LSTM layers
    output_size = 2  # ax, ay as output

    # create forecasting model
    model = MyModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, output_size=output_size,
                    device= args.device)

    # optimizer and schedular
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-3)
    lr_scheduler = None
    if args.step_size is not None:
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args.step_size)
    start_epoch = 0

    # # Loss Criterion
    # criterion = L2Loss(col_wt=args.col_wt, col_distance=args.col_distance) if args.loss == 'L2' \
    #                 else PredictionLoss(col_wt=args.col_wt, col_distance=args.col_distance)
    #
    criterion = torch.nn.MSELoss()

    # train
    if args.load_state:
        # load pretrained model.
        # useful for transfer learning
        print("Loading Model Dict")
        with open(args.load_state, 'rb') as f:
            checkpoint = torch.load(f)
        pretrained_state_dict = checkpoint['state_dict']
        model.load_state_dict(pretrained_state_dict, strict=args.load_state_strict)

        if args.load_full_state:
            # load optimizers from last training
            # useful to continue model training
            print("Loading Optimizer Dict")
            optimizer.load_state_dict(checkpoint['optimizer'])
            lr_scheduler.load_state_dict(checkpoint['scheduler'])
            start_epoch = checkpoint['epoch']

    # trainer
    trainer = Trainer(model, optimizer=optimizer, lr_scheduler=lr_scheduler, device=args.device,
                      criterion=criterion, batch_size=args.batch_size, obs_length=args.obs_length,
                      pred_length=args.pred_length, augment=args.augment, normalize_scene=args.normalize_scene,
                      save_every=args.save_every, start_length=args.start_length, obs_dropout=args.obs_dropout,
                      augment_noise=args.augment_noise, val_flag=val_flag)
    trainer.loop(train_scenes, val_scenes, train_goals, val_goals, args.output, epochs=args.epochs, start_epoch=start_epoch)

In [4]:
TrackRow = namedtuple('Row', ['frame', 'car_id', 'x', 'y', 'xVelocity', 'yVelocity','prediction_number', 'scene_id'])
TrackRow.__new__.__defaults__ = (None, None, None, None, None, None, None, None)
SceneRow = namedtuple('Row', ['scene', 'car_id', 'start', 'end', 'fps', 'tag'])
SceneRow.__new__.__defaults__ = (None, None, None, None, None, None)

In [5]:

def drop_distant(xyv, r=10):
    """
    Drops pedestrians more than r meters away from primary ped
    """
    distance_2 = np.sum(np.square(xyv[:, :, :2] - xyv[:, 0:1, :2]), axis=2)
    mask = np.nanmin(distance_2, axis=0) < r ** 2
    return xyv[:, mask], mask


def rotate_path(path, theta):
    ct = math.cos(theta)
    st = math.sin(theta)

    return [TrackRow(r.frame, r.car_id, ct * r.x + st * r.y, -st * r.x + ct * r.y,
                     ct*r.xVelocity + st * r.yVelocity, -st * r.xVelocity + ct * r.yVelocity
                     ) for r in path]


def random_rotation_of_paths(paths):
    theta = random.random() * 2.0 * math.pi
    return [rotate_path(path, theta) for path in paths]


def random_rotation(xyv, goals=None):
    theta = random.random() * 2.0 * math.pi
    ct = math.cos(theta)
    st = math.sin(theta)
    r = np.array([[ct, st], [-st, ct]])

    # 旋转位置
    rotated_positions = np.einsum('ptc,ci->pti', xyv[:, :, :2], r)

    # 旋转速度
    rotated_velocities = np.einsum('ptc,ci->pti', xyv[:, :, 2:], r)

    # 合并旋转后的位置和速度
    rotated_xyv = np.concatenate((rotated_positions, rotated_velocities), axis=2)

    if goals is None:
        return rotated_xyv
    else:
        # 如果提供了goals，则旋转goals（假设它们也是[x, y, xVelocity, yVelocity]格式的）
        rotated_goals_pos = np.einsum('tc,ci->ti', goals[:, :2], r)
        rotated_goals_vel = np.einsum('tc,ci->ti', goals[:, 2:], r)
        rotated_goals = np.concatenate((rotated_goals_pos, rotated_goals_vel), axis=1)
        return rotated_xyv, rotated_goals


def theta_rotation(xyv, theta):
    ct = math.cos(theta)
    st = math.sin(theta)
    r = np.array([[ct, st], [-st, ct]])

    # 旋转位置
    rotated_positions = np.einsum('ptc,ci->pti', xyv[:, :, :2], r)

    # 旋转速度
    rotated_velocities = np.einsum('ptc,ci->pti', xyv[:, :, 2:], r)

    # 合并旋转后的位置和速度
    rotated_xyv = np.concatenate((rotated_positions, rotated_velocities), axis=2)

    return rotated_xyv


def shift(xyv, center):
    # theta = random.random() * 2.0 * math.pi
    xyv = xyv - center[np.newaxis, np.newaxis, :]
    return xyv


def center_scene(xyv, obs_length=9, car_id=0, goals=None):
    if goals is not None:
        goals = goals[np.newaxis, :, :]
    # Center
    center = xyv[obs_length-1, car_id]  # Last Observation
    xyv = shift(xyv, center)
    if goals is not None:
        goals = shift(goals, center)
    # Rotate
    last_obs = xyv[obs_length-1, car_id]
    second_last_obs = xyv[obs_length-2, car_id]
    diff = np.array([last_obs[0] - second_last_obs[0], last_obs[1] - second_last_obs[1],
                        last_obs[2] - second_last_obs[2], last_obs[3] - second_last_obs[3]])
    theta = np.arctan2(diff[1], diff[0])
    rotation = -theta + np.pi/2
    xyv = theta_rotation(xyv, rotation)
    if goals is not None:
        goals = theta_rotation(goals, rotation)
        return xyv, rotation, center, goals[0]
    return xyv, rotation, center


def inverse_scene(xyv, rotation, center):
    xyv = theta_rotation(xyv, -rotation)
    xyv = shift(xyv, -center)
    return xyv


def drop_unobserved(xyv, obs_length=25):
    loc_at_obs = xyv[obs_length-1]
    absent_at_obs = np.isnan(loc_at_obs).any(axis=1)
    mask = ~absent_at_obs
    return xyv[:, mask], mask


def neigh_nan(xyv):
    return np.isnan(xyv).all()


def add_noise(observation, thresh=0.005, obs_length=25, ped='primary'):
    if ped == 'primary':
        observation[:obs_length, 0] += np.random.uniform(-thresh, thresh, observation[:obs_length, 0].shape)
    elif ped == 'neigh':
        observation[:obs_length, 1:] += np.random.uniform(-thresh, thresh, observation[:obs_length, 1:].shape)
    else:
        raise ValueError

    return observation


In [6]:

class Reader(object):
    """Read trajnet files.

    :param scene_type: None -> numpy.array, 'rows' -> TrackRow and SceneRow, 'paths': grouped rows (primary pedestrian first), 'tags': numpy.array and scene tag
    :param image_file: Associated image file of the scene
    """
    def __init__(self, input_file, scene_type=None, image_file=None):
        if scene_type is not None and scene_type not in {'rows', 'paths', 'tags'}:
            raise Exception('scene_type not supported')
        self.scene_type = scene_type

        self.tracks_by_frame = defaultdict(list)
        self.scenes_by_id = dict()
        self.scaler = None
        self.read_file(input_file)


    def read_file(self, input_file):
        with open(input_file, 'r') as f:
            for line in f:
                line = json.loads(line)
                track = line.get('track')
                if track is not None:
                    row = TrackRow(track['f'], track['c'], track['x'], track['y'], track['xVelocity'], track['yVelocity'],
                                   track.get('prediction_number'), track.get('scene_id'))
                    self.tracks_by_frame[row.frame].append(row)
                    continue

                scene = line.get('scene')
                if scene is not None:
                    row = SceneRow(scene['id'], scene['c'], scene['s'], scene['e'],
                                   scene.get('fps'), scene.get('tag'))
                    self.scenes_by_id[row.scene] = row

    def scenes(self, randomize=False, limit=0, ids=None, sample=None):
        scene_ids = self.scenes_by_id.keys()
        if ids is not None:
            scene_ids = ids
        if randomize:
            scene_ids = list(scene_ids)
            random.shuffle(scene_ids)
        if limit:
            scene_ids = itertools.islice(scene_ids, limit)
        if sample is not None:
            scene_ids = random.sample(scene_ids, int(len(scene_ids) * sample))
        for scene_id in scene_ids:
            yield self.scene(scene_id)

    @staticmethod
    def track_rows_to_paths(primary_car, track_rows):
        primary_path = []
        other_paths = defaultdict(list)
        for row in track_rows:
            if row.car_id == primary_car:
                primary_path.append(row)
                continue
            other_paths[row.car_id].append(row)

        return [primary_path] + list(other_paths.values())

    @staticmethod
    def paths_to_xyv(paths):
        """Convert paths to numpy array with nan as blanks."""
        frames = set(r.frame for r in paths[0])
        cars = set(row.car_id
                          for path in paths
                          for row in path if row.frame in frames)
        paths = [path for path in paths if path[0].car_id in cars]
        frames = sorted(frames)
        cars = list(cars)

        frame_to_index = {frame: i for i, frame in enumerate(frames)}
        xyv = np.full((len(frames), len(cars), 4), np.nan)

        for car_index, path in enumerate(paths):
            for row in path:
                if row.frame not in frame_to_index:
                    continue
                entry = xyv[frame_to_index[row.frame]][car_index]
                entry[0] = row.x
                entry[1] = row.y
                entry[2] = row.xVelocity
                entry[3] = row.yVelocity

        return xyv

    def scene(self, scene_id):
        scene = self.scenes_by_id.get(scene_id)
        if scene is None:
            raise Exception('scene with that id not found')

        frames = range(scene.start, scene.end + 1)
        track_rows = [r
                      for frame in frames
                      for r in self.tracks_by_frame.get(frame, [])]

        # return as rows
        if self.scene_type == 'rows':
            return scene_id, scene.id, track_rows

        # return as paths
        paths = self.track_rows_to_paths(scene.car_id, track_rows)
        if self.scene_type == 'paths':
            return scene_id, paths

        # return with scene tag
        if self.scene_type == 'tags':
            return scene_id, scene.tag, self.paths_to_xyv(paths)

        # return a numpy array
        return scene_id, self.paths_to_xyv(paths)


In [7]:
class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, device, scale_factors=None):
        super(MyModel, self).__init__()
        self.l1 = nn.Linear(input_size, output_size)  # Linear layer
        self.tanh = nn.Tanh()
        self.l2 = nn.Linear(output_size, output_size)
        self.lstm = nn.LSTM(output_size, hidden_size, num_layers)  # LSTM layer
        self.reset_parameters()
        self.scale_factors = scale_factors \
            if scale_factors is not None else \
                torch.tensor([2, 0.5, 0.01, 0.02]).to(device) # scaler for ind
        self.device = device


#                 torch.tensor([2, 0.5, 0.01, 0.02]).to(device) # scaler for ind
#                 torch.tensor([4.0, 0.3, 0.5, 0.015]).to(device) #scaler for highd
    def reset_parameters(self):
        # Reset parameters for linear and LSTM layers
        self.l1.reset_parameters()
        self.l2.reset_parameters()
        self.lstm.reset_parameters()

    def encoder(self, input_seq, lengths):
        fx_fy = self.l1(input_seq)
        fx_fy = self.tanh(fx_fy)
        fx_fy = self.l2(fx_fy)

        # Sort the sequences by length in descending order
        lengths, indices = lengths.sort(descending=True)
        fx_fy = fx_fy[:, indices]

        # Pack the sequences
        fx_fy_packed = pack_padded_sequence(fx_fy, lengths.cpu(), enforce_sorted=False)

        # Get the outputs from the LSTM
        # _, (acc, _) = self.lstm(fx_fy_packed)
        output, _ = self.lstm(fx_fy_packed)
        output_padded, _ = pad_packed_sequence(output, batch_first=True)
        acc = output_padded[:, -1]

        # Reshape the output
        acc = acc.squeeze(0)

        # Restore the original order
        _, inv_indices = indices.sort()
        acc = acc[inv_indices]

        return acc

    def decoder(self, last_positions, acc_tensor):
        fps = 25
        dt = 1 / fps  # Time step

        # Extract x, y, x_velocity, y_velocity from the last positions
        x, y, x_velocity, y_velocity = torch.chunk(last_positions, 4, dim=-1)

        # Compute predicted velocities
        x_velocity_pred = x_velocity + dt * acc_tensor[:, 0].unsqueeze(1)
        y_velocity_pred = y_velocity + dt * acc_tensor[:, 1].unsqueeze(1)

        # Compute predicted positions
        x_pred = x + dt * x_velocity_pred
        y_pred = y + dt * y_velocity_pred

        # Combine the predicted positions and velocities
        pred_frame = torch.cat([x_pred, y_pred, x_velocity_pred, y_velocity_pred], dim=-1)

        return pred_frame

    def forward(self, observed, prediction_truth, batch_split):

        # Unscale the observed data
        unscaled_observed = observed * self.scale_factors.unsqueeze(0).unsqueeze(0)

        # get all nan lines indices
        mask = torch.isnan(unscaled_observed[:, :, 0])
        nan_rows = mask.all(dim=0)
        indices_to_keep = torch.where(~nan_rows)[0]

        # Delete the all nan lines
        unscaled_observed = torch.index_select(unscaled_observed, 1, indices_to_keep)
        prediction_truth = torch.index_select(prediction_truth, 1, indices_to_keep)

        # Update the batch_split
        nan_indices = torch.nonzero(nan_rows).squeeze()
        for i in range(1, len(batch_split)):
            subtract_count = (nan_indices < batch_split[i]).sum().item()
            batch_split[i] -= subtract_count

        # Mask the observed data
        mask = torch.isnan(unscaled_observed[:, :, 0])
        unscaled_observed = torch.where(torch.isnan(unscaled_observed), torch.zeros_like(unscaled_observed),
                                        unscaled_observed)
        lengths = (~mask).sum(dim=0).cpu().long()
        pooled_positions = self.generate_pooling(unscaled_observed, batch_split)

        # Get each batch's primary_car for truth without loop
        primary_indices = batch_split[:-1]
        target_truth = prediction_truth[:, primary_indices].to(self.device)
        target_pred = torch.zeros_like(target_truth).to(self.device)
        pred_frames = target_truth.shape[0]
        pred_full = torch.zeros_like(prediction_truth).to(self.device)
        acc = torch.zeros_like(target_truth[:, :, :2]).to(self.device)
        for i in range(pred_frames):
            last_positions = pooled_positions[-1, :, :4]

            acc_tensor = self.encoder(pooled_positions, lengths)

            # Decode the acc_tensor to get the predicted positions and velocities for the current frame
            pred_frame = self.decoder(last_positions, acc_tensor)

            # Scale the predictions
            scaled_prediction = pred_frame / self.scale_factors.unsqueeze(0)

            # Store the prediction 应该是此frame对应的batch数量行，因为是primary cars。列数是4.
            target_pred[i] = scaled_prediction[primary_indices]

            # Generate pooling for the predicted frame
            pooled_pred_frame = self.generate_pooling(pred_frame, batch_split)

            # Add the new pooled_pred_frame to the end of pooled_positions and remove the first frame
            pooled_positions = torch.cat([pooled_positions[1:], pooled_pred_frame.unsqueeze(0)], dim=0)
            
            # generate full prediciton to make the visualization possible
            pred_full[i] = scaled_prediction
            
            acc[i] = acc_tensor[primary_indices]
        truth_full = prediction_truth
            

        return target_pred, target_truth, pred_full, truth_full, acc

#     def generate_pooling(self, obs_data, batch_split):
#         single_frame = False

#         # Check if it's a single frame data
#         if len(obs_data.shape) == 2:
#             single_frame = True
#             obs_data = obs_data.unsqueeze(0)  # Add a frame dimension

#         num_frames, num_cars, _ = obs_data.shape
#         curr_positions = obs_data.clone()  # Clone obs_data to get curr_positions

#         # Create a mask where cars with all zeros are considered missing
#         valid_cars = (obs_data.sum(dim=2) != 0).unsqueeze(2)

#         # Expand dimensions to allow broadcasting
#         expanded_obs = obs_data.unsqueeze(2)
#         expanded_valid_cars = valid_cars.unsqueeze(2)

#         # Calculate the differences using broadcasting
#         diffs = expanded_obs - obs_data.unsqueeze(1)

#         # Use the valid cars mask to ignore calculations involving missing cars
#         diffs *= expanded_valid_cars
# #         non_zero = (diffs.all(dim=2) != 0)

#         # Sum the differences along the car axis
#         rel_positions = diffs.sum(dim=2)

#         # Concatenate along the third dimension
#         pooled_positions = torch.cat((curr_positions, rel_positions), dim=2)

#         # Remove the added frame dimension if it's a single frame data
#         if single_frame:
#             pooled_positions = pooled_positions.squeeze(0)

#         return pooled_positions

    def generate_pooling(self, obs_data, batch_split):
        single_frame = False

        # Check if it's a single frame data
        if len(obs_data.shape) == 2:
            single_frame = True
            obs_data = obs_data.unsqueeze(0)  # Add a frame dimension

        num_frames, num_cars, _ = obs_data.shape
        curr_positions = obs_data.clone()  # Clone obs_data to get curr_positions

        # Create a mask where cars with all zeros are considered missing
        valid_cars = (obs_data.sum(dim=2) != 0).unsqueeze(2)

        # Expand dimensions to allow broadcasting
        expanded_obs = obs_data.unsqueeze(2)
        expanded_valid_cars = valid_cars.unsqueeze(2)

        # Calculate the differences using broadcasting
        diffs = expanded_obs - obs_data.unsqueeze(1)

        eps = 1e-10
        diffs[diffs == 0] += eps  # Add a very small value to zero entries

        # Calculate the inverse
        diffs_inv = 1.0 / diffs

        # Set values that are extremely large (because of the division by a small value) to zero
        threshold = 1e9  # You can adjust this threshold as needed
        diffs_inv[diffs_inv.abs() > threshold] = 0

        # Use the valid cars mask to ignore calculations involving missing cars
        diffs_inv *= expanded_valid_cars

        # Sum the inversed differences along the car axis
        scores = diffs_inv.sum(dim=2)

        # Concatenate along the third dimension
        pooled_positions = torch.cat((curr_positions, scores), dim=2)

        # Remove the added frame dimension if it's a single frame data
        if single_frame:
            pooled_positions = pooled_positions.squeeze(0)

        return pooled_positions



class LSTMPredictor(object):
    def __init__(self, model):
        self.model = model

    def save(self, state, filename):
        with open(filename, 'wb') as f:
            torch.save(self, f)

        # # during development, good for compatibility across API changes:
        # # Save state for optimizer to continue training in future
        with open(filename + '.state', 'wb') as f:
            torch.save(state, f)

    @staticmethod
    def load(filename):
        with open(filename, 'rb') as f:
            return torch.load(f)

    def __call__(self, paths, scene_goal, n_predict=12, modes=1, predict_all=True, obs_length=25, start_length=0,
                 args=None):
        self.model.eval()
        # self.model.train()
        with torch.no_grad():
            xyv = Reader.paths_to_xyv(paths)
            # xyv = add_noise(xyv, thresh=args.thresh, ped=args.ped_type)
            batch_split = [0, xyv.shape[1]]

            if args.normalize_scene:
                xyv, rotation, center, scene_goal = center_scene(xyv, obs_length, goals=scene_goal)

            xyv = torch.Tensor(xyv)  # .to(self.device)
            scene_goal = torch.Tensor(scene_goal)  # .to(device)
            batch_split = torch.Tensor(batch_split).long()

            multimodal_outputs = {}
            for num_p in range(modes):
                # _, output_scenes = self.model(xyv[start_length:obs_length], scene_goal, batch_split, xyv[obs_length:-1].clone())
                _, output_scenes = self.model(xyv[start_length:obs_length], scene_goal, batch_split, n_predict=n_predict)
                output_scenes = output_scenes.numpy()
                if args.normalize_scene:
                    output_scenes = inverse_scene(output_scenes, rotation, center)
                output_primary = output_scenes[-n_predict:, 0]
                output_neighs = output_scenes[-n_predict:, 1:]
                ## Dictionary of predictions. Each key corresponds to one mode
                multimodal_outputs[num_p] = [output_primary, output_neighs]

        ## Return Dictionary of predictions. Each key corresponds to one mode
        return multimodal_outputs

In [8]:
device = torch.device('cpu')
# hyper parameters
input_size = 8  # frame, car_id, x, y, xVelocity, yVelocity, dx, dy, dvx, dvy
hidden_size = 2  # LSTM hidden size
num_layers = 2  # LSTM layers
output_size = 2  # ax, ay as output

# create forecasting model
model = MyModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, output_size=output_size,
                device=device)
model.to(device)
criterion = nn.MSELoss().to(device)

In [9]:
## Initialize batch of scenes
batch_size = 10
batch_scene = []
batch_split = [0]

for scene_i, (filename, scene_id, paths) in enumerate(train_scenes):
    ## make new scene
    scene = Reader.paths_to_xyv(paths)

    #scene: frame_index x car_index x xyv
    ## Augment scene to batch of scenes
    batch_scene.append(scene)
    batch_split.append(int(scene.shape[1]))

    if ((scene_i + 1) % batch_size == 0) or ((scene_i + 1) == len(train_scenes)):
        ## Construct Batch torch.Size([50, 317, 4])
        batch_scene = np.concatenate(batch_scene, axis=1)
        batch_split = np.cumsum(batch_split)
        batch_scene = torch.Tensor(batch_scene).to(device)
        batch_split = torch.Tensor(batch_split).to(device).long()
        observed = batch_scene[0:25].clone()
        prediction_truth = batch_scene[25:50].clone()
        # 25frames x 10primary cars x [x, y, xV, yV]
        train_pred, train_truth = model(observed, prediction_truth, batch_split)
        print('train_pred:\n',train_pred,train_pred.shape)
        print('train_truth:\n',train_truth,train_truth.shape)
        loss = criterion(train_pred, train_truth)
        print('loss:\n', loss)
        break
        ## Reset Batch
        batch_scene = []
        batch_scene_goal = []
        batch_split = [0]


NameError: name 'train_scenes' is not defined

In [ ]:
## Prepare data
test_scenes, test_goals, _ = prepare_data(path='DATA_BLOCK/ind' , subset='/test/', sample=1, goals=0)
#for train_scenes: file, scene_id, scene. scene = 50 * car_ids * 4

In [ ]:
def load_model(model, optimizer=None, path='default_path', model_type='vanilla', output='default_output', return_epoch=False, return_losses=False):
    # Construct the paths based on the manually provided parameters
    model_file_path = 'OUTPUT_BLOCK/{}/lstm_{}_{}.pkl'.format(path, model_type, output)
    state_file_path = 'OUTPUT_BLOCK/{}/lstm_{}_{}.state'.format(path, model_type, output)
    
    # Load the model architecture and weights
    loaded_model = torch.load(model_file_path)
    model.load_state_dict(loaded_model.model.state_dict())

    # Load the model's state (e.g. optimizer state, epoch, losses)
    checkpoint = torch.load(state_file_path)

    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    results = []
    if return_epoch:
        epoch = checkpoint.get('epoch', None)
        results.append(epoch)
    if return_losses:
        train_losses = checkpoint.get('train_losses', [])
        val_losses = checkpoint.get('val_losses', [])
        results.extend([train_losses, val_losses])

    return results if len(results) > 0 else None


In [ ]:
# train_scale=torch.tensor([4.0, 0.3, 0.5, 0.015]).to(device)
train_scale=torch.tensor([2, 0.5, 0.01, 0.02]).to(device)

In [ ]:
load_model(model, path='ind1', output='None')

In [ ]:
def Traj_plot(scene_index, train_scenes, train_scale, model, criterion, device):
    model.eval()

    # Extract specific scene from train_scenes based on scene_index
    filename, scene_id, paths = train_scenes[scene_index]
    scene = Reader.paths_to_xyv(paths)
    scene_tensor = torch.Tensor(scene).to(device)
    observed = scene_tensor[0:25]
    prediction_truth = scene_tensor[25:50]

    # Create a virtual batch
    num_copies = 3
    observed_batch = observed.repeat(1, num_copies, 1)
    prediction_truth_batch = prediction_truth.repeat(1, num_copies, 1)
    
    # Compute the original batch_split
    original_batch_split = torch.tensor([observed.shape[1]]).to(device)
    batch_split = original_batch_split.repeat(num_copies)
    cumulative_batch_split = torch.cumsum(batch_split, dim=0)

    # Compute the predictions
    with torch.no_grad():
        train_pred_batch, train_truth_batch, pred_full_batch, truth_full_batch, acc = model(observed_batch, prediction_truth_batch, cumulative_batch_split)

    # Use the first result from the virtual batch
    train_pred = train_pred_batch[:, 1]
    train_truth = train_truth_batch[:, 1]
    acc = acc[:, 1].cpu().numpy()

    pred_full = pred_full_batch
    truth_full = truth_full_batch
    
    
    # Calculate the loss
    train_loss = criterion(train_pred.squeeze()[:, :4], train_truth.squeeze()[:, :4])

    # Convert to numpy for plotting
    observed_np = (observed_batch * train_scale).cpu().numpy()
    train_pred_np = (train_pred * train_scale).cpu().numpy()
    train_truth_np = (train_truth * train_scale).cpu().numpy()
    
    pred_full_np = (pred_full * train_scale.unsqueeze(0).unsqueeze(0)).cpu().numpy()
    truth_full_np = (truth_full * train_scale.unsqueeze(0).unsqueeze(0)).cpu().numpy()

    # Plotting
    plt.figure(figsize=(12, 8))
    plt.plot(observed_np[:, 0, 0], observed_np[:, 0, 1], 'b--', linewidth=2, label='Traj_observed')
    plt.scatter(observed_np[:, 0, 0], observed_np[:, 0, 1], c='b', s=5)
    
    plt.plot(train_pred_np[:, 0], train_pred_np[:, 1], 'g-', label='Traj_pred')
    plt.scatter(train_pred_np[:, 0], train_pred_np[:, 1], c='g', s=5)

    plt.plot(train_truth_np[:, 0], train_truth_np[:, 1], 'r-', label='Traj_target')
    plt.scatter(train_truth_np[:, 0], train_truth_np[:, 1], c='r', s=5)

    plt.scatter(observed_np[-1, 0, 0], observed_np[-1, 0, 1], marker='^', s=100, c='black', label='Current State')
    train_loss = train_loss.item()
    plt.text(0.95, 0.01, f'Loss: {train_loss:.2f}', verticalalignment='bottom', horizontalalignment='right', transform=plt.gca().transAxes)

    
#     ### others
#     plt.plot(pred_full_np[:, 1:, 0], pred_full_np[:, 1:, 1], 'y-')
#     plt.scatter(pred_full_np[:, 1:, 0], pred_full_np[:, 1:, 1], c='y', s=5)

#     plt.plot(truth_full_np[:, 1:, 0], truth_full_np[:, 1:, 1], 'r-')
#     plt.scatter(truth_full_np[:, 1:, 0], truth_full_np[:, 1:, 1], c='r', s=5)

    # 使用 quiver 来添加加速度向量
    plt.quiver(train_pred_np[:, 0], 
               train_pred_np[:, 1], 
               acc[:, 0], 
               acc[:, 1], 
               angles='xy', 
               scale_units='xy', 
               scale=1, 
               color='y',
               width=0.003,
               headwidth=3, 
               headlength=4, 
               headaxislength=3.5,
               label='force'
              )
    
    plt.axis('equal') 
    plt.legend()
    plt.savefig('Traj_plot.png')
    plt.show()



In [ ]:
Traj_plot(400, test_scenes, train_scale, model, criterion, device)